Trong bài toán này, chúng ta sẽ được cho một bộ dataset mô tả thông tin về nhân viên trong một công ty, bao gồm các features liên quan đến nhân viên và mức lương của họ. Nhiệm vụ của chúng ta là phân tích, xử lí bộ data dưới đây và trả lời các câu hỏi yêu cầu người làm phải thực hiện coding.  
[employee_data.csv](https://drive.google.com/file/d/1WkVx5tinsxuU3SnIwLNiYU7X2Sqo3vgG/view?usp=sharing)

## **Thực hiện các yêu cầu sau đây**
1. **Đọc dữ liệu**  
Sử dụng pandas, đọc file csv được cung cấp, sau đó hiển thị ra màn hỉnh để hiểu các trường dữ liệu.
2. **Label Encoding**  
Chuyển đổi các cột dữ liệu dạng chữ (cụ thể là cột "Gender" và "Position") sang dạng số bằng cách sử dụng `LabelEncoder` từ thư viện `sklearn`.  
3. **Tách dữ liệu thành bộ feature (X) và label (y)**
- Sử dụng các cột "Gender", "Experience (Years)" và "Position" làm features đầu vào (`X`).
- Sử dụng cột "Salary" làm biến đầu ra (`y`).  
4. **Tách tập dữ liệu thành tập train và test**  
- Chia dữ liệu thành tập huấn luyện (`X_train`, `y_train`) và tập kiểm tra (`X_test`, `y_test`) với tỷ lệ `80:20`.
- Đảm bảo rằng việc chia tách dữ liệu là ngẫu nhiên nhưng tái lập (reproducibility) được với `random_state=42`

# Thư viện

In [1]:
# Các thư viện cần thiết
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Tải và đọc dữ liệu

In [2]:
# Tải dữ liệu
# https://drive.google.com/file/d/1WkVx5tinsxuU3SnIwLNiYU7X2Sqo3vgG/view?usp=sharing
!gdown 1WkVx5tinsxuU3SnIwLNiYU7X2Sqo3vgG

Downloading...
From: https://drive.google.com/uc?id=1WkVx5tinsxuU3SnIwLNiYU7X2Sqo3vgG
To: c:\Users\admin\OneDrive\Máy tính\Exam_AIO2024\AIO_2024_note\Module_3\Source Codes\RForest\employee_data.csv

  0%|          | 0.00/14.2k [00:00<?, ?B/s]
100%|██████████| 14.2k/14.2k [00:00<00:00, 1.19MB/s]


In [3]:
# Đọc dữ liệu
data = pd.read_csv('employee_data.csv')
data.head()

,ID,Gender,Experience (Years),Position,Salary
0,1,F,4,DevOps Engineer,109976
1,2,M,6,DevOps Engineer,120088
2,3,M,17,Web Developer,181301
3,4,M,7,Systems Administrator,77530
4,5,F,13,Systems Administrator,152397


In [4]:
data.describe()

,ID,Experience (Years),Salary
count,400.000000,400.000000,400.00000
mean,200.500000,9.670000,131701.19750
std,115.614301,6.101571,43351.50899
min,1.000000,0.000000,43643.00000
25%,100.750000,4.000000,100484.75000
50%,200.500000,10.000000,128561.50000
75%,300.250000,15.000000,157735.00000
max,400.000000,20.000000,269950.00000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  400 non-null    int64 
 1   Gender              400 non-null    object
 2   Experience (Years)  400 non-null    int64 
 3   Position            400 non-null    object
 4   Salary              400 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 15.8+ KB


# Tiền xử lý

In [6]:
# Mã hóa giới tính và vị trí công việc
label_encoder_gender = LabelEncoder()
label_encoder_position = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data['Position'] = label_encoder_position.fit_transform(data['Position'])

X = data.drop(columns=['ID', 'Salary'])
y = data['Salary']

In [7]:
print(X.head())

   Gender  Experience (Years)  Position
0       0                   4         2
1       1                   6         2
2       1                  17        10
3       1                   7         8
4       0                  13         8


In [8]:
print(y.head())

0    109976
1    120088
2    181301
3     77530
4    152397
Name: Salary, dtype: int64


In [9]:
# Chia tập dữ liệu thành tập train và test với tỉ lệ (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((320, 3), (320,), (80, 3), (80,))

# Khởi tạo các tham số của mô hình Random Forest

In [10]:
# Định nghĩa các tham số cho mô hình Random Forest
params = {
    'n_estimators': 50,         # Số lượng cây trong rừng
    # 'max_depth': 3,             # Độ sâu tối đa của mỗi cây
    # 'min_samples_split': 2,     # Số lượng mẫu tối thiểu để chia nhánh
    # 'min_samples_leaf': 1,      # Số lượng mẫu tối thiểu tại mỗi nút lá
    'random_state': 42          # Giá trị ngẫu nhiên để đảm bảo kết quả lặp lại
}

# Huấn luyện mô hình Random Forest

In [11]:
# Huấn luyện mô hình
model = RandomForestRegressor(**params)  # Khởi tạo mô hình Random Forest với các tham số đã định nghĩa
model.fit(X_train, y_train)  # Huấn luyện mô hình trên tập huấn luyện

RandomForestRegressor(n_estimators=50, random_state=42)

In [12]:
# Dự đoán trên tập test
y_pred = model.predict(X_test)  # Dự đoán giá trị trên tập test dựa trên mô hình đã huấn luyện

# Đánh giá mô hình Random Forest

In [13]:
# Đánh giá mô hình với MSE và R2
origin_mse = mean_squared_error(y_test, y_pred)
origin_r2 = r2_score(y_test, y_pred)

print(f'MSE: {origin_mse}')
print(f'R-squared: {origin_r2}')

MSE: 827087272.7854073
R-squared: 0.5572421510566792


### Thử nghiệm Number of Trees (n_estimators)

In [14]:
min_mse = float('inf')
best_n_trees = 0

for n_trees in [10, 20, 50, 100]:
    rf_model = RandomForestRegressor(n_estimators=n_trees, random_state=42)
    rf_model.fit(X_train, y_train)
    prediction = rf_model.predict(X_test)
    mse = mean_squared_error(y_test, prediction)
    r2 = r2_score(y_test, prediction)
    print(f'{n_trees} trees, R^2 Score: {r2:.4f}, MSE: {mse:.4f}')

    if mse < min_mse:
        min_mse = mse
        best_n_trees = n_trees

print(f'\nBest number of trees: {best_n_trees} with MSE: {min_mse:.4f}')

10 trees, R^2 Score: 0.5630, MSE: 816412296.1290
20 trees, R^2 Score: 0.5673, MSE: 808325351.4423
50 trees, R^2 Score: 0.5572, MSE: 827087272.7854
100 trees, R^2 Score: 0.5541, MSE: 833009696.9098

Best number of trees: 20 with MSE: 808325351.4423


## Tree Depth (Độ sâu của cây)

In [15]:
min_mse = float('inf')
best_depth = 0

for depth in range(1,11):
    rf_model = RandomForestRegressor(n_estimators=100, max_depth=depth, random_state=42)
    rf_model.fit(X_train, y_train)
    prediction = rf_model.predict(X_test)
    mse = mean_squared_error(y_test, prediction)
    r2 = r2_score(y_test, prediction)
    print(f'Max Depth {depth}, R^2 Score: {r2:.4f}, MSE: {mse:.4f}')

    if mse < min_mse:
        min_mse = mse
        best_depth = depth

print(f'\nBest depth: {best_depth} with MSE: {min_mse:.4f}')

Max Depth 1, R^2 Score: 0.4078, MSE: 1106206160.4543
Max Depth 2, R^2 Score: 0.5754, MSE: 793159725.1288
Max Depth 3, R^2 Score: 0.6273, MSE: 696285217.0212
Max Depth 4, R^2 Score: 0.6431, MSE: 666786934.1791
Max Depth 5, R^2 Score: 0.6507, MSE: 652425054.1957
Max Depth 6, R^2 Score: 0.6376, MSE: 676956494.2709
Max Depth 7, R^2 Score: 0.6134, MSE: 722183057.7126
Max Depth 8, R^2 Score: 0.5859, MSE: 773566420.4845
Max Depth 9, R^2 Score: 0.5698, MSE: 803670730.0819
Max Depth 10, R^2 Score: 0.5571, MSE: 827439285.5817

Best depth: 5 with MSE: 652425054.1957
